In [3]:
# Ler arquivo CSV referente aos pagamentos

%pyspark
rdd_clients = spark.read.format(
   "com.databricks.spark.csv").option(
   "header", "true").option(
   "inferSchema", "true").option(
   "delimiter", ',').load(
   'clients.csv')

rdd_clients.count()

unknown magic command 'pyspark'
UnknownMagic: unknown magic command 'pyspark'



In [2]:
%pyspark
# Verificando Metadados da tabela
rdd_clients.printSchema()

UsageError: Line magic function `%pyspark` not found.


In [ ]:
%pyspark
# Verificando 5 linhas da tabela
rdd_clients.show(5)

In [ ]:
# Ler arquivo CSV referente aos pagamentos

%pyspark
rdd_payments = spark.read.format(
   "com.databricks.spark.csv").option(
   "header", "true").option(
   "inferSchema", "true").option(
   "delimiter", ',').load(
   'payments.csv')

rdd_payments.count()

In [ ]:
%pyspark
rdd_payments.printSchema()

In [ ]:
%pyspark
rdd_payments.show(15)

In [ ]:
# Ler arquivo CSV referente a empréstimos contratados pelos clientes

%pyspark
rdd_loan = spark.read.format(
   "com.databricks.spark.csv").option(
   "header", "true").option(
   "inferSchema", "true").option(
   "delimiter", ',').load(
   'loans.csv')
rdd_loan.count()

In [ ]:
%pyspark
rdd_loan.printSchema()

In [ ]:
%pyspark
rdd_loan.show(5)

In [ ]:
# Disponibilizando todas as tabelas RDDs para SQLContext

%pyspark
payments_dfm = rdd_payments.registerTempTable("payments_dfm")
clients_dfm = rdd_clients.registerTempTable("clients_dfm")
loans_dfm = rdd_loan.registerTempTable("loans_dfm")

In [ ]:
# Vamos investigar o cliente id 46109

%pyspark
spark.sql(
             """SELECT 
                    *,
                    cast(loan_start as date) as DT_INICIO,
                    cast(loan_end as date) as DT_FIM,
                    datediff(cast(loan_end as date),cast(loan_start as date)) as QT_DIAS_START_END,
                    round(datediff(cast(loan_end as date),cast(loan_start as date))/30.5) as QT_MESES_START_END,
                    round(datediff(cast(loan_end as date),cast(loan_start as date))/365) as QT_ANOS_START_END
                FROM
                    loans_dfm 
                WHERE 
                    client_id = 46109 and loan_type in ('home') 
                order by cast(loan_start as date)    
             """
         ).show()


In [ ]:
# Vamos investigar o contrato Id 10243 do cliente 46109 na base de pagamentos

%pyspark
spark.sql(
             """SELECT 
                    *,
                    cast(payment_date as date ) as DT_PGTO
                    --min(payment_date) as DT_PRIMEIRO_PAGAMENTO,
                    --max(payment_date) as DT_ULTIMO_PAGAMENTO,
                FROM
                    payments_dfm 
                WHERE 
                    loan_id = 10243 
             """
         ).show()

In [ ]:
# Vamos investigar os pagamentos referentes ao contrato 10243 do cliente 46109

%pyspark
spark.sql(
             """SELECT
                    loan_id,
                    sum(case when missed = 1 then 0 else payment_amount end) as VL_PGTO_MISS,
                    sum(case when missed = 0 then 0 else payment_amount end) as VL_PGTO_NOT_MISS,
                    sum(payment_amount) as VL_TOT,
                    cast(min(payment_date) as date) as DT_PRIMEIRO_PAGAMENTO,
                    cast(max(payment_date) as date) as DT_ULTIMO_PAGAMENTO,
                    count(*) as QT_PGTOS
                FROM
                    payments_dfm 
                WHERE 
                    loan_id = 10243
                GROUP BY  
                    loan_id
             """
         ).show()


In [ ]:
# Aplicar algumas Transformações nos dados

%pyspark
clients_00 = spark.sql(
                     """SELECT 
                            *,
                            round(log(income),4) as VL_LOG_RENDA,
                            cast(joined as date) as DT_JOINED,
                            substr(cast(joined as date),6,2) as VL_MES_JOINED
                        FROM
                            clients_dfm                     
                     """
                     )
clients_00_dfm = clients_00.registerTempTable("clients_00_dfm")                     
clients_00.show(15)

In [ ]:
# Calcular o valor médio, mínimo e máximo de empréstimos para cada cliente

%pyspark
loans_00 = spark.sql(
                     """SELECT 
                            client_id,
                            round(avg(loan_amount),4) as VL_MED_EMP_CLI,
                            min(loan_amount) as VL_MIN_EMP_CLI,
                            max(loan_amount) as VL_MAX_EMP_CLI
                        FROM
                            loans_dfm  
                        group by client_id     
                     """
                     )
loans_00_dfm = loans_00.registerTempTable("loans_00_dfm")                         
loans_00.show(5)

In [ ]:
# Vamos levar as informações calculadas para a tabela de clientes

%pyspark
clientes_01 = spark.sql(
                     """SELECT 
                            a.*,
                            b.VL_MED_EMP_CLI,
                            b.VL_MIN_EMP_CLI,
                            b.VL_MAX_EMP_CLI
                        FROM
                            clients_00_dfm as a  
                        left join  loans_00_dfm as b
                        on a.client_id = b.client_id
                     """
                     )
                     
clientes_01.show(5)

In [ ]:
# Verificar os pagamentos dos empréstimos efetuados

%pyspark
loans_02 = spark.sql(
                     """SELECT 
                            a.*,
                            b.payment_amount,
                            b.payment_date,
                            b.missed
                        FROM
                            loans_dfm as a  
                        left join  payments_dfm as b
                        on a.loan_id = b.loan_id
                     """
                     )
loans_02_dfm = loans_02.registerTempTable("loans_02_dfm")                       
loans_02.show(5)


In [ ]:
# Quantidade de pagamentos por tipo de empréstimo(Granularidade Contrato)

%pyspark
loans_03 = spark.sql(
                     """SELECT 
                            loan_type,
                            round(avg(payment_amount),2) as VL_MED_PAGAMENTOS,
                            count(*) as QT_CONTRATOS_TIPO
                        FROM
                            loans_02_dfm 
                        group by 1  
                     """
                     )
loans_03_dfm = loans_03.registerTempTable("loans_03_dfm")                       
loans_03.show()

In [ ]:
# Calculo MED, MIN e MAX de pagamentos para cada tipo de emprestimo (Granularidade CPF)

%pyspark
loans_04 = spark.sql(
                     """SELECT 
                            client_id,
                            sum(case when loan_type in ('cash') then 1 else 0 end) as QT_EMP_CASH,
                            avg(case when loan_type in ('cash') then loan_amount else 0 end) as VL_MEDEMP_CASH,
                            count(*) as QT_CONTRATOS_TIPO
                        FROM
                            loans_02_dfm 
                        group by 1  
                     """
                     )
loans_04_dfm = loans_04.registerTempTable("loans_04_dfm")                       
loans_04.show()

In [ ]:
# Tratamento de datas sistêmicas

%pyspark
import time
from datetime import datetime
from datetime import date, timedelta
dataref = datetime.now()-timedelta(0, 0, 0, 0, 0, +2)
anomesdia = dataref.strftime("%Y%m%d")
dataref
#print('Data Hoje: ',dataref.strftime("%Y%m%d"))

In [ ]:
# Salvar tabela com particionamento por data de referência

%pyspark
etl_final = spark.sql(
                     """SELECT 
                            *,
                            '{}' as PK_DATREF
                        FROM
                            loans_04_dfm 
                     """.format(anomesdia)
                     )
             
etl_final.show()

In [ ]:
# Salvando Parquet não particionado com HDFS (Não visível ao Hive)

%pyspark
etl_final.write.parquet("etl_final_00.parquet")

In [ ]:
# Salvando parquet não particionado no Hive

%pyspark
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

# Salvando a tabela como temporária no Hive
#hive_context.registerDataFrameAsTable(etl_final, "etl_final_00")

# Salvando a tabela no schema (database) treinamento como permanente do Hive (modo append)
etl_final.write.mode('append').saveAsTable("treinamento.etl_final_00")

hive_context.sql("show tables in treinamento").show()

In [ ]:
%pyspark
spark.catalog.dropTempView("clients_00_dfm") 
spark.catalog.dropTempView("clients_dfm") 
spark.catalog.dropTempView("loans_00_dfm") 
spark.catalog.dropTempView("loans_02_dfm") 
spark.catalog.dropTempView("loans_03_dfm") 
spark.catalog.dropTempView("loans_04_dfm") 
spark.catalog.dropTempView("loans_dfm") 
spark.catalog.dropTempView("payments_dfm") 

In [ ]:
%pyspark
hive_context.sql("show tables in treinamento").show()

In [ ]:
%pyspark
tab_hive = hive_context.sql("select * from treinamento.etl_final_00").show()

In [ ]:
# Salvando parquet particionado utilizando Hive

%pyspark
# Salvando a tabela no schema (database) treinamento como permanente do Hive (modo append) particionado
etl_final.write.mode('append').partitionBy('PK_DATREF').saveAsTable("treinamento.etl_final_01")

hive_context.sql("show tables in treinamento").show()

In [ ]:
%pyspark
# Ler partição Hive
df_hive_parquet = sqlContext.read.option("mergeSchema", "true").parquet("/user/hive/warehouse/treinamento.db/etl_final_01/PK_DATREF=20181126")
df_hive_parquet.count()

In [ ]:
%pyspark
df_hive_parquet.show()

In [ ]:
# Salvando parquet particionado no S3 (AWS)

%pyspark
nm_path_s3 = 's3://treinamento-big-data/'
etl_final.write.partitionBy('PK_DATREF').parquet(nm_path_s3, mode='append')